In [ ]:
!pip install transformers==4.46.2 datasets==3.1.0 torch==2.5.1+cu121 sklearn-pandas==2.2.0 numpy==1.26.4 fvcore==0.1.5.post20221221

In [ ]:
# !pip list | grep fvcore

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from fvcore.nn import FlopCountAnalysis
import numpy as np
import pandas as pd
import datasets
import torch
import time
import os

The WANDB_API_KEY is nessesary to use the Trainer class from HuggingFace and is what's used in the example notebook. However, I think we may all have to put in our own api key secret into google colab for it to work.

In [ ]:
from google.colab import userdata
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
langs = ['java', 'python', 'pharo']

# langs = ['java'] # Using Java as the only language for testing purposes.
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

training = True

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

java_train-00000-of-00001.parquet:   0%|          | 0.00/680k [00:00<?, ?B/s]

java_test-00000-of-00001.parquet:   0%|          | 0.00/174k [00:00<?, ?B/s]

python_train-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

...

Generating java_test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Generating python_train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

Generating python_test split:   0%|          | 0/406 [00:00<?, ? examples/s]

Generating pharo_train split:   0%|          | 0/1298 [00:00<?, ? examples/s]

Generating pharo_test split:   0%|          | 0/289 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [ ]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [ ]:
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [ ]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
    return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [ ]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float32")))
  processed_dataset.set_format('pt')
  return processed_dataset

Evaluating the predictions.

Below is non-functional code I still haven't finnished. We need to implement a way for the logits returned from the model to be converted into binary (0 - 1) predictions for the labels, and for those labels to be compaired to the ground truth labels of the testing datasets for each language. The competition also wants us to complete this step using profiling that they have set up, and which is partially shown below. The link to the reference notebook for the competition is [here](https://colab.research.google.com/drive/1RULzasjO_nrqiXLrGze-PznFlHKtGQ4s?usp=sharing). Also, another notebook that is fairly close to what I believe we are trying to do can be found [here](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb).

In [ ]:
# scores must be global so that it can later be referenced for printing the results
scores = pd.DataFrame(columns=['lan', 'cat', 'precision', 'recall', 'f1'])

In [ ]:
def compute_metrics(eval_pred, lang, categories):
    global scores
    scores.drop(scores[scores['lan'] == lang].index, inplace=True)

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame(metrics_list)
    scores = pd.concat([scores, temp_scores], ignore_index=True)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [ ]:
def compute_metrics_validate(eval_pred, lang, categories):

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame(metrics_list)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [ ]:
def measure_runtime_and_flops(trainer, test_dataset):
  print(f' \n** Evaluating runtime and flops for {lang} test set. **\n')
  total_time = 0
  total_flops = 0

  for i in range(10):
    for batch in test_dataset:
        inputs = {key: val.unsqueeze(0).to(trainer.model.device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
        with torch.profiler.profile(with_flops=True, activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
            start_time = time.time()
            _ = trainer.model(**inputs)
            end_time = time.time()

        total_time += (end_time - start_time)
        total_flops += sum(k.flops for k in prof.key_averages()) / 1e9

    avg_runtime = total_time / 10
    avg_flops = total_flops / 10
    return avg_runtime, avg_flops, total_flops, total_time

In [ ]:
# Wrapper is needed because model can only take eval_pred for compute_metrics, but lang and labels are necessary for output
def compute_metrics_wrapper(eval_pred):
  if training:
    return compute_metrics_validate(eval_pred, lang, labels[lang])
  else:
    print(f'\n ** Evaluating metrics for {lang} test set. ** \n')
    return compute_metrics(eval_pred, lang, labels[lang])


In [ ]:
hyperparams = {
    'java': {
        'epochs': 20,
        'lr': 5e-5,
        'weight_decay': 0.01,
        'batch_size': 8
    },
    'python': {
        'epochs': 20,
        'lr': 5e-5,
        'weight_decay': 0.01,
        'batch_size': 8
    },
    'pharo': {
        'epochs': 20,
        'lr': 5e-5,
        'weight_decay': 0.01,
        'batch_size': 8
    }
}

In [ ]:
total_flops = 0
total_time = 0
metrics_dict = {}

# Main Training Loop
for lang in langs:

    training = True

    num_labels = len(labels[lang])
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    dataset = preprocess_dataset(ds[f'{lang}_train'])
    test_dataset = preprocess_dataset(ds[f'{lang}_test'])

    train_val_split = dataset.train_test_split(test_size=0.2, seed=27)
    train_dataset = train_val_split['train']
    validation_dataset = train_val_split['test']

    training_args = TrainingArguments(
        output_dir=f'./results_{lang}',
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f'./logs_{lang}',
        per_device_train_batch_size=hyperparams[lang]['batch_size'],
        per_device_eval_batch_size=hyperparams[lang]['batch_size'],
        num_train_epochs=hyperparams[lang]['epochs'],
        weight_decay=hyperparams[lang]['weight_decay'],
        learning_rate=hyperparams[lang]['lr'],
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        seed=27
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        processing_class=tokenizer,
        compute_metrics=compute_metrics_wrapper,
    )

    print(f'\n** Training {lang} model. **\n')
    trainer.train()
    trainer.evaluate()

    training = False

    test_trainer = Trainer(
        model=trainer.model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        processing_class=tokenizer,
        compute_metrics=compute_metrics_wrapper,
    )

    test_trainer.evaluate()

    avg_runtime, avg_flops, total_flops, total_time = measure_runtime_and_flops(trainer, test_dataset)

    metrics_dict[lang] = {
        'avg_runtime': avg_runtime,
        'avg_flops': avg_flops,
        'total_flops': total_flops,
        'total_time': total_time
    }

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7614 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7614 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

...

Casting the dataset:   0%|          | 0/1298 [00:00<?, ? examples/s]

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/289 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.287300,0.296838,0.223734,0.358761,0.189070
2,0.262100,0.231524,0.455018,0.592069,0.382423
3,0.178200,0.221423,0.520315,0.600077,0.468488
4,0.113200,0.202693,0.620347,0.878775,0.557129
5,0.122000,0.208424,0.667731,0.787967,0.597673
6,0.057600,0.213306,0.653555,0.795521,0.589163
7,0.037400,0.241754,0.645974,0.782498,0.593432
8,0.025600,0.230932,0.664402,0.757172,0.619860
9,0.030000,0.240900,0.689723,0.781157,0.640122
10,0.019100,0.238468,0.678752,0.771343,0.638274



 ** Evaluating metrics for pharo test set. ** 

 
** Evaluating runtime and flops for pharo test set. **



In [ ]:
scores

,lan,cat,precision,recall,f1
0,java,summary,0.890351,0.910314,0.900222
1,java,Ownership,1.000000,1.000000,1.000000
2,java,Expand,0.427083,0.401961,0.414141
3,java,usage,0.941645,0.823666,0.878713
4,java,Pointer,0.730612,0.972826,0.834499
5,java,deprecation,0.909091,0.666667,0.769231
6,java,rational,0.461538,0.352941,0.400000
7,python,Usage,0.847619,0.735537,0.787611
8,python,Parameters,0.824000,0.804688,0.814229
9,python,DevelopmentNotes,0.358974,0.341463,0.350000


In [ ]:
try:
  scores.to_csv('/content/drive/MyDrive/CS_440_Experiment_Results/scores_1.csv')
  print('Saved scores.')
except:
  print('Could not save scores.')

Saved scores.


In [ ]:
for lang in metrics_dict:
  print(f'{lang}: {metrics_dict[lang]}')

java: {'avg_runtime': 3.4694892883300783, 'avg_flops': 3751.3437552001515, 'total_flops': 37513.43755200152, 'total_time': 34.69489288330078}
python: {'avg_runtime': 0.8293900728225708, 'avg_flops': 882.924840268792, 'total_flops': 8829.24840268792, 'total_time': 8.293900728225708}
pharo: {'avg_runtime': 0.5908348321914673, 'avg_flops': 628.4859972479973, 'total_flops': 6284.859972479972, 'total_time': 5.908348321914673}


In [ ]:
total_flops = 0
total_time = 0
avg_runtime = 0
avg_flops = 0

for lang in metrics_dict:
  total_flops += metrics_dict[lang]['total_flops']
  total_time += metrics_dict[lang]['total_time']
  avg_runtime += metrics_dict[lang]['avg_runtime']
  avg_flops += metrics_dict[lang]['avg_flops']

In [ ]:
print(f'Total Flops: {total_flops}')
print(f'Total Time: {total_time}')
print(f'Average Flops: {avg_flops}')
print(f'Average Time: {avg_runtime}')

f1_mean = scores['f1'].mean()
print(f'Average F1: {f1_mean}')

Total Flops: 52627.545927169405
Total Time: 48.89714193344116
Average Flops: 5262.75459271694
Average Time: 4.889714193344116
Average F1: 0.6779750123660004


In [ ]:
max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = scores.f1.mean()

round(score(avg_f1, avg_runtime, avg_flops), 2)

0.4